In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/Shareddrives/si370/final_delivery/Pilot_Test/

/content/drive/Shareddrives/si370/final_delivery/Pilot_Test


Prepare training data

In [ ]:
import pandas as pd
sasb_data = pd.read_csv("Data/training_data/sasb_full_training.csv")
sasb_data.drop(["Unnamed: 0"], axis=1, inplace=True)
labels = sasb_data.Sector.unique()

label_dict = {}
for index, label in enumerate(sorted(labels)):
    label_dict[label] = index
label_dict

{'consumer goods': 0,
 'extractives & minerals processing': 1,
 'financials': 2,
 'food & beverage': 3,
 'health care': 4,
 'infrastructure': 5,
 'renewable resources & alternative energy': 6,
 'resource transformation': 7,
 'services': 8,
 'technology & communications': 9,
 'transportation': 10}

In [ ]:
sasb_data['label'] = sasb_data.Sector.replace(label_dict)

Split data into train/evaluation and test sets


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sasb_data.index.values, 
                                                  sasb_data.label.values, 
                                                  test_size=0.05, 
                                                  random_state=42, 
                                                  stratify=sasb_data.label.values)

sasb_data['data_type'] = ['not_set']*sasb_data.shape[0]

sasb_data.loc[X_train, 'data_type'] = 'train'
sasb_data.loc[X_test, 'data_type'] = 'test'

sasb_data.groupby(['Sector', 'label', 'data_type']).count()


train_text
Sector                                   label data_type            
consumer goods                           0     test               14
                                               train             265
extractives & minerals processing        1     test               13
                                               train             248
financials                               2     test               11
                                               train             215
food & beverage                          3     test                8
                                               train             151
health care                              4     test               11
                                               train             202
infrastructure                           5     test               27
                                               train             509
renewable resources & alternative energy 6     test                2
                                               train              32
resource transformation                  7     test               21
                                               train             401
services                                 8     test                7
                                               train             127
technology & communications              9     test               15
                                               train             288
transportation                           10    test               12
                                               train             222

In [ ]:
sasb_train_eval = sasb_data[sasb_data["data_type"] == "train"]
sasb_test = sasb_data[sasb_data["data_type"] == "test"]

In [ ]:
print(len(sasb_train_eval))
print(len(sasb_test))

2660
141


In [ ]:
sasb_train_eval

,train_text,Sector,label,data_type
0,development distribution and manufacture of a ...,health care,4,train
1,the worldwide development manufacture and dist...,health care,4,train
2,201 847 6800 All references herein to the Comp...,health care,4,train
3,201 847 6800 All references herein to the Comp...,health care,4,train
4,201 847 6800 All references in this Form 10 K ...,health care,4,train
...,...,...,...,...
2796,along with our joint venture partners are the ...,food & beverage,3,train
2797,along with our joint venture partners are the ...,food & beverage,3,train
2798,in 2014 in Delaware without its subsidiaries 8...,extractives & minerals processing,1,train
2799,in 2014 in Delaware without its subsidiaries q...,extractives & minerals processing,1,train


Split train/evaluation set into two sets

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(sasb_train_eval.index.values, 
                                                  sasb_train_eval.label.values, 
                                                  test_size=0.1, 
                                                  random_state=42, 
                                                  stratify=sasb_train_eval.label.values)

sasb_train_eval['data_type'] = ['not_set']*sasb_train_eval.shape[0]

sasb_train_eval.loc[X_train, 'data_type'] = 'train'
sasb_train_eval.loc[X_val, 'data_type'] = 'val'

sasb_train_eval.groupby(['Sector', 'label', 'data_type']).count()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


train_text
Sector                                   label data_type            
consumer goods                           0     train             239
                                               val                26
extractives & minerals processing        1     train             223
                                               val                25
financials                               2     train             193
                                               val                22
food & beverage                          3     train             136
                                               val                15
health care                              4     train             182
                                               val                20
infrastructure                           5     train             458
                                               val                51
renewable resources & alternative energy 6     train              29
                                               val                 3
resource transformation                  7     train             361
                                               val                40
services                                 8     train             114
                                               val                13
technology & communications              9     train             259
                                               val                29
transportation                           10    train             200
                                               val                22

Tokenize data and prepare dataset for training

In [ ]:
!pip install transformers
import torch

from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

     |████████████████████████████████| 2.5 MB 8.6 MB/s 
     |████████████████████████████████| 3.3 MB 48.2 MB/s 
     |████████████████████████████████| 895 kB 47.9 MB/s 


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    sasb_train_eval[sasb_train_eval.data_type=='train'].train_text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    max_length=384, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    sasb_train_eval[sasb_train_eval.data_type=='val'].train_text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    max_length=384, 
    return_tensors='pt'
)

encoded_data_test = tokenizer.batch_encode_plus(
    sasb_test.train_text.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    max_length=384, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(sasb_train_eval[sasb_train_eval.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(sasb_train_eval[sasb_train_eval.data_type=='val'].label.values)

input_ids_test = encoded_data_test['input_ids']
attention_masks_test = encoded_data_test['attention_mask']
labels_test = torch.tensor(sasb_test.label.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)
dataset_test = TensorDataset(input_ids_test, attention_masks_test, labels_test)

BERT pre-trained model

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 3

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

dataloader_test = DataLoader(dataset_test, 
                                   sampler=SequentialSampler(dataset_test), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
import random
import numpy as np
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)



In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)


cuda


In [ ]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        print("During training one batch, the loss of the output is:", loss)
        print("Loss.item:", loss.item())
        logits = outputs[1]
        print("During training one batch, the logits of the output is:", logits)
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Performs a single optimization step (parameter update).
        optimizer.step()
        # You call scheduler.step() every batch, right after optimizer.step(), to update the learning rate.
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'second_finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')


Epoch 1
Training loss: 1.0508883071171684
During training one batch, the loss of the output is: tensor(0.0663, device='cuda:0')
Loss.item: 0.06630364805459976
During training one batch, the logits of the output is: tensor([[-0.6865, -0.8005,  0.1129, -0.1187,  4.8091, -1.2620, -0.7044,  0.3684,
         -1.1163, -0.0608,  0.0755],
        [-0.5453, -0.8886, -0.0203, -0.7657, -0.0732, -0.2819, -0.0276, -0.4421,
         -0.2821,  5.3303, -0.9420],
        [-0.6616, -0.4183,  4.2310, -0.4848,  0.6458, -0.3895, -0.6974, -0.8123,
         -0.7116, -0.2900,  0.1634]], device='cuda:0')
During training one batch, the loss of the output is: tensor(0.0978, device='cuda:0')
Loss.item: 0.09782391041517258
During training one batch, the logits of the output is: tensor([[-0.6639, -0.4180,  4.2361, -0.4892,  0.6438, -0.3837, -0.6977, -0.8172,
         -0.7074, -0.2841,  0.1546],
        [ 0.2844, -0.2100, -0.4576,  4.3961, -0.1006, -0.8701, -0.3782, -0.0558,
         -0.8827, -0.4972,  0.1545],
   


Epoch 2
Training loss: 0.1899838079208214
During training one batch, the loss of the output is: tensor(0.0106, device='cuda:0')
Loss.item: 0.010575339198112488
During training one batch, the logits of the output is: tensor([[-0.8802, -1.0575, -0.2376, -0.5923,  6.5041, -1.0575, -0.5482,  0.3080,
         -1.0764, -0.3656, -0.3453],
        [-0.7761, -0.8334, -0.3644, -0.8552, -0.3385, -0.5625, -0.0344, -0.4794,
         -0.4332,  7.1737, -1.0443],
        [-1.4345, -0.4258,  6.0115, -1.2174, -0.2932,  1.2439, -0.8951, -1.2289,
         -1.1962, -0.9744, -0.4086]], device='cuda:0')
During training one batch, the loss of the output is: tensor(0.0133, device='cuda:0')
Loss.item: 0.013289575465023518
During training one batch, the logits of the output is: tensor([[-1.4404, -0.4261,  6.0154, -1.2216, -0.2941,  1.2466, -0.8949, -1.2353,
         -1.1917, -0.9678, -0.4099],
        [-0.3379, -0.1994, -0.6676,  6.4729, -0.1613, -0.8580, -0.1293, -0.3541,
         -0.8133, -0.5412,  0.1045],
 


Epoch 3
Training loss: 0.10497941752664633
During training one batch, the loss of the output is: tensor(0.0039, device='cuda:0')
Loss.item: 0.0038989621680229902
During training one batch, the logits of the output is: tensor([[-0.9326, -1.0005, -0.2449, -0.6353,  7.1609, -1.0428, -0.5020, -0.1358,
         -0.9132, -0.3481, -0.5821],
        [-0.7205, -0.7060, -0.3395, -0.8514, -0.5987, -0.5420, -0.0350, -0.7630,
         -0.5424,  7.8038, -1.1204],
        [-1.1393, -0.6189,  6.9851, -1.0930, -0.3011,  0.5451, -0.8737, -1.6412,
         -1.1224, -0.7912, -0.7378]], device='cuda:0')
During training one batch, the loss of the output is: tensor(0.0052, device='cuda:0')
Loss.item: 0.0052443030290305614
During training one batch, the logits of the output is: tensor([[-1.1523, -0.6141,  6.9847, -1.1001, -0.3010,  0.5523, -0.8772, -1.6411,
         -1.1219, -0.7840, -0.7413],
        [-0.4987, -0.1645, -0.6696,  7.1588, -0.3103, -0.8290,  0.0400, -0.5914,
         -0.6873, -0.4809, -0.0616]


Epoch 4
Training loss: 0.06131058626818146
During training one batch, the loss of the output is: tensor(0.0028, device='cuda:0')
Loss.item: 0.0027760800439864397
During training one batch, the logits of the output is: tensor([[-0.9249, -1.1167, -0.2159, -0.7373,  7.5265, -1.0910, -0.5532, -0.0764,
         -0.8483, -0.5105, -0.5103],
        [-0.6667, -0.7899, -0.3544, -0.9142, -0.7136, -0.6251, -0.0889, -0.8650,
         -0.4038,  8.1427, -1.0952],
        [-1.1769, -0.8190,  7.2943, -1.1201, -0.3192,  0.7019, -0.9097, -1.8109,
         -1.1423, -0.8159, -0.7682]], device='cuda:0')
During training one batch, the loss of the output is: tensor(0.0037, device='cuda:0')
Loss.item: 0.003712427569553256
During training one batch, the logits of the output is: tensor([[-1.1959e+00, -8.0791e-01,  7.2925e+00, -1.1241e+00, -3.1469e-01,
          7.0400e-01, -9.1777e-01, -1.8013e+00, -1.1510e+00, -8.1429e-01,
         -7.6494e-01],
        [-4.1153e-01, -2.4291e-01, -6.2505e-01,  7.5039e+00, -3.


Epoch 5
Training loss: 0.048088318735558334
During training one batch, the loss of the output is: tensor(0.0024, device='cuda:0')
Loss.item: 0.002357172081246972
During training one batch, the logits of the output is: tensor([[-0.9193, -1.1662, -0.2090, -0.7000,  7.6527, -1.0946, -0.5439, -0.1330,
         -0.9061, -0.4784, -0.5357],
        [-0.7113, -0.8246, -0.3583, -0.9164, -0.6891, -0.6408, -0.1011, -0.8243,
         -0.4418,  8.2758, -1.0999],
        [-1.2134, -0.9247,  7.4367, -1.1273, -0.2718,  0.5534, -0.8972, -1.7185,
         -1.1419, -0.8066, -0.7819]], device='cuda:0')
During training one batch, the loss of the output is: tensor(0.0031, device='cuda:0')
Loss.item: 0.003148164600133896
During training one batch, the logits of the output is: tensor([[-1.2322, -0.9149,  7.4358, -1.1322, -0.2670,  0.5547, -0.9042, -1.7077,
         -1.1475, -0.8058, -0.7812],
        [-0.4013, -0.2715, -0.6599,  7.6403, -0.3560, -0.9039, -0.0276, -0.6661,
         -0.6195, -0.5579, -0.1319],

Load the fine-tuned model 

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)
model.load_state_dict(torch.load('second_finetuned_BERT_epoch_5.model', map_location=torch.device('cpu')))

<All keys matched successfully>

Take a look in some data in the validation set

In [ ]:
dataiter = iter(dataloader_validation)

texts, _, labels = dataiter.next()

In [ ]:
labels

tensor([4, 9, 2])

In [ ]:
output_test = model(texts.to(device))

In [ ]:
output_test

SequenceClassifierOutput([('logits',
                           tensor([[-0.9503, -1.1377, -0.2512, -0.7353,  7.6463, -1.0559, -0.5708, -0.0470,
                                    -0.9613, -0.5182, -0.5091],
                                   [-0.6774, -0.7376, -0.5423, -1.0054, -0.7230, -0.6022, -0.1937, -0.6914,
                                    -0.4369,  8.2002, -0.9837],
                                   [-1.1941, -0.8498,  7.3005, -1.2992, -0.3343,  0.9047, -1.0476, -1.7439,
                                    -1.2216, -0.8222, -0.6837]], device='cuda:0', grad_fn=<AddmmBackward>))])

In [ ]:
_, predicted = torch.max(output_test[0], 1)

In [ ]:
predicted

tensor([4, 9, 2], device='cuda:0')

The predicted labels are the same with the original labels

Evaluate the test set

In [ ]:
val_loss, predictions, true_vals = evaluate(dataloader_test)
val_f1 = f1_score_func(predictions, true_vals)

During training one batch, the loss of the output is: tensor(0.0028, device='cuda:0')
Loss.item: 0.002779792994260788
During training one batch, the logits of the output is: tensor([[-0.9532, -1.1742, -0.2287, -0.7632,  7.6411, -1.0676, -0.5676,  0.0294,
         -0.8962, -0.6260, -0.5330],
        [-0.9438, -1.1612, -0.2293, -0.7412,  7.6482, -1.0868, -0.5661, -0.0167,
         -0.8913, -0.5956, -0.5216],
        [-0.4132, -0.2427, -0.6596,  7.6271, -0.4221, -0.8534, -0.0496, -0.6764,
         -0.5776, -0.5732, -0.1509]], device='cuda:0')
During training one batch, the loss of the output is: tensor(0.0023, device='cuda:0')
Loss.item: 0.0023189960047602654
During training one batch, the logits of the output is: tensor([[-5.1296e-01, -2.3827e-01, -6.7461e-01,  7.6346e+00, -3.6074e-01,
         -8.5894e-01, -1.9362e-03, -6.2375e-01, -6.1870e-01, -4.9537e-01,
         -1.7561e-01],
        [-1.3333e-01, -5.2672e-01, -1.6201e+00, -1.1057e+00, -1.1939e+00,
         -1.2865e+00, -1.1629e+00,

Our fune-tuned model has 0.9716165220328032 f1 score on the testset

In [ ]:
val_f1

0.9716165220328032

Test the model with some random texts on the internet

Test moderna description from their website

In [ ]:
moderna = """The Ann Arbor Area Transportation Authority (TheRide) operates the public transit system for the greater Ann Arbor-Ypsilanti area. TheRide provides reliable, safe, affordable transportation services. We offer a variety of service types including: Fixed route buses Paratransit services (A-Ride) Commuter services (ExpressRide, Park & Ride, Vanpools) Demand-responsive services (FlexRide, NightRide, HolidayRide) Event services (FootballRide)  Airport services (AirRide)"""

In [ ]:
moderna_test = [moderna]
moderna_test

['The Ann Arbor Area Transportation Authority (TheRide) operates the public transit system for the greater Ann Arbor-Ypsilanti area. TheRide provides reliable, safe, affordable transportation services. We offer a variety of service types including: Fixed route buses Paratransit services (A-Ride) Commuter services (ExpressRide, Park & Ride, Vanpools) Demand-responsive services (FlexRide, NightRide, HolidayRide) Event services (FootballRide)  Airport services (AirRide)']

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_test = tokenizer.batch_encode_plus(
    moderna_test, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    padding=True, 
    max_length=384, 
    return_tensors='pt'
)

input_ids_moderna = encoded_data_test['input_ids']
attention_masks_moderna = encoded_data_test['attention_mask']
import numpy as np
labels_test = torch.tensor(np.array([4]))

In [ ]:
output_moderna = model(input_ids_moderna.to(device))
_, predicted = torch.max(output_moderna[0], 1)
predicted

tensor([5], device='cuda:0')